In [1]:
!pip3 install -q transformers datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx 1.8.0 requires pyarrow<6,>=1, but you have pyarrow 11.0.0 which is incompatible.
tfx-bsl 1.8.0 requires pyarrow<6,>=1, but you have pyarrow 11.0.0 which is incompatible.
tensorflow-transform 1.8.0 requires pyarrow<6,>=1, but you have pyarrow 11.0.0 which is incompatible.
tensorflow-model-analysis 0.39.0 requires pyarrow<6,>=1, but you have pyarrow 11.0.0 which is incompatible.
tensorflow-data-validation 1.8.0 requires pyarrow<6,>=1, but you have pyarrow 11.0.0 which is incompatible.
apache-beam 2.40.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.40.0 requires pyarrow<8.0.0,>=0.15.1, but you have pyarrow 11.0.0 which is incompatible.
allennlp 2.9.0 requires transformers<4.16,>=4.1, but you have transformers 4.26.1 which is incompatible.


In [2]:
import os
import json
import math
import glob
import shutil
import traceback

import numpy as np
import pandas as pd

from copy import deepcopy
from datasets import Dataset
from datetime import datetime
from transformers import EvalPrediction
from datasets.dataset_dict import DatasetDict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

2023-03-17 09:52:53.659263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-17 09:52:53.659289: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
transcript_folder = "./../trans/"
input_file = "data_and_corr_pvals.xlsx"
input_data_sheet = "data"

In [5]:
def get_file_content(file_path):
    file_content = None
    with open(file_path, "r") as fp:
        file_content = fp.read()
    return file_content

In [6]:
data = pd.read_excel(input_file, sheet_name=input_data_sheet)
if "Unnamed: 0" in data.columns: # Removing the index column
    data.drop("Unnamed: 0", axis=1, inplace=True)

In [7]:
cols_to_keep = ["memory_change", "PALFAMS28Percentile", "mapped_user_id"]
data.drop(list(set(list(data.columns)).difference(set(cols_to_keep))), axis=1, inplace=True)

In [8]:
files = os.listdir(transcript_folder)
relevant_files = []
for each_file in files:
    complete_path = "{0}{1}".format(transcript_folder, each_file)
    if complete_path.endswith(".txt") and os.stat(complete_path).st_size > 0:
        relevant_files.append(complete_path)
print("Only {0} files have data out of {1}".format(len(relevant_files), len(files)))
del(files)

Only 2374 files have data out of 3006


In [9]:
final_data_list = []
for each_file in relevant_files:
    file_content = get_file_content(each_file)
    user_id = int(each_file.split("_")[2])
    try:
        each_data = {
                        "user_id": user_id,
                        "file_path": each_file,
                        "file_content": file_content,
                        "memory_change": data[data["mapped_user_id"]==user_id]["memory_change"].iloc[0]
                    }
        final_data_list.append(each_data)
    except Exception as e:
        pass

In [10]:
final_data = pd.DataFrame(final_data_list)
if "Unnamed: 0" in final_data.columns: # Removing the index column
    final_data.drop("Unnamed: 0", axis=1, inplace=True)
del(final_data_list)

In [14]:
final_data.to_excel("final_data.xlsx", header=True, index=False)
data.to_excel("intermediate_data.xlsx", header=True, index=False)